<a href="https://colab.research.google.com/github/SoumyaAIDev/Hugging_Face_LLM_models/blob/main/LLama2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets accelerate torch
!pip install datasets
!pip install huggingface_hub
!huggingface-cli login


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          

In [ ]:
from datasets import load_dataset

# Load the dataset (replace with your dataset name)
dataset = load_dataset("ASSERT-KTH/DISL",'decomposed')

# Select only 10,000 samples from the train split
train_dataset = dataset["train"].shuffle(seed=42).select(range(10000))

# Remove unnecessary columns, keeping only "contract_address" and "source_code"
filtered_train_dataset = train_dataset.remove_columns(
    [col for col in train_dataset.column_names if col not in ["contract_address", "source_code"]]
)

# Print the first example to verify
print(filtered_train_dataset[0])

# If you have validation or test splits, you can process them similarly:
if "validation" in dataset:
    validation_dataset = dataset["validation"].shuffle(seed=42).select(range(1000))  # Example for 1,000 samples
    filtered_validation_dataset = validation_dataset.remove_columns(
        [col for col in validation_dataset.column_names if col not in ["contract_address", "source_code"]]
    )
    print(filtered_validation_dataset[0])


Resolving data files:   0%|          | 0/18 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/18 [00:00<?, ?it/s]

{'contract_address': '0x9c4aaa5e284b4271e8213d727fcd33afcb9c8604', 'source_code': '// OpenZeppelin Contracts (last updated v4.5.0) (token/ERC721/ERC721.sol)\r\n\r\npragma solidity ^0.8.0;\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n/**\r\n * @dev Implementation of https://eips.ethereum.org/EIPS/eip-721[ERC721] Non-Fungible Token Standard, including\r\n * the Metadata extension, but not including the Enumerable extension, which is available separately as\r\n * {ERC721Enumerable}.\r\n */\r\ncontract ERC721 is Context, ERC165, IERC721, IERC721Metadata {\r\n    using Address for address;\r\n    using Strings for uint256;\r\n\r\n    // Token name\r\n    string private _name;\r\n\r\n    // Token symbol\r\n    string private _symbol;\r\n\r\n    // Mapping from token ID to owner address\r\n    mapping(uint256 => address) private _owners;\r\n\r\n    // Mapping owner address to token count\r\n    mapping(address => uint256) private _balances;\r\n\r\n    // Mapping from token ID to approved address\r\n   

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B")
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})  # Add a [PAD] token
    print("Added a [PAD] token to the tokenizer.")

# Load the dataset
dataset = load_dataset("ASSERT-KTH/DISL",'decomposed')  # Replace with your dataset name

# Select only 10,000 samples from the train split and keep required columns
train_dataset = dataset["train"].shuffle(seed=42).select(range(10000))
filtered_dataset = train_dataset.remove_columns(
    [col for col in train_dataset.column_names if col not in ["contract_address", "source_code"]]
)

# Preprocessing function
def preprocess_function(examples):
    inputs = tokenizer(
        examples["contract_address"],  # Input column
        padding="max_length",
        truncation=True,
        max_length=128
    )
    outputs = tokenizer(
        examples["source_code"],  # Output column
        padding="max_length",
        truncation=True,
        max_length=512
    )
    labels = [
        [(token if token != tokenizer.pad_token_id else -100) for token in label]
        for label in outputs["input_ids"]
    ]
    return {
        "input_ids": inputs["input_ids"],
        "attention_mask": inputs["attention_mask"],
        "labels": labels
    }

# Apply preprocessing
tokenized_dataset = filtered_dataset.map(preprocess_function, batched=True)

# Verify tokenized dataset
print(tokenized_dataset[0])


Added a [PAD] token to the tokenizer.


Resolving data files:   0%|          | 0/18 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/18 [00:00<?, ?it/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

{'contract_address': '0x9c4aaa5e284b4271e8213d727fcd33afcb9c8604', 'source_code': '// OpenZeppelin Contracts (last updated v4.5.0) (token/ERC721/ERC721.sol)\r\n\r\npragma solidity ^0.8.0;\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n/**\r\n * @dev Implementation of https://eips.ethereum.org/EIPS/eip-721[ERC721] Non-Fungible Token Standard, including\r\n * the Metadata extension, but not including the Enumerable extension, which is available separately as\r\n * {ERC721Enumerable}.\r\n */\r\ncontract ERC721 is Context, ERC165, IERC721, IERC721Metadata {\r\n    using Address for address;\r\n    using Strings for uint256;\r\n\r\n    // Token name\r\n    string private _name;\r\n\r\n    // Token symbol\r\n    string private _symbol;\r\n\r\n    // Mapping from token ID to owner address\r\n    mapping(uint256 => address) private _owners;\r\n\r\n    // Mapping owner address to token count\r\n    mapping(address => uint256) private _balances;\r\n\r\n    // Mapping from token ID to approved address\r\n   

In [ ]:
# Remove unnecessary columns
columns_to_keep = ["contract_address","source_code","input_ids", "attention_mask", "labels"]
tokenized_dataset = tokenized_dataset.remove_columns(
    [col for col in tokenized_dataset.column_names if col not in columns_to_keep]
)


In [ ]:
# Split dataset into train and validation sets
split_dataset = tokenized_dataset.train_test_split(test_size=0.1)
train_dataset = split_dataset["train"]
eval_dataset = split_dataset["test"]


In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=8,  # Adjust batch size based on memory
    num_train_epochs=3,
    weight_decay=0.01,
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=10,
    save_total_limit=2,
)


In [ ]:
from transformers import Trainer, AutoModelForCausalLM

# Load the model
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B")

# Define the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
)


<ipython-input-32-cbab5b9313d5>:7: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [4]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B")
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})  # Add a [PAD] token
    print("Added a [PAD] token to the tokenizer.")


# Load dataset
dataset = load_dataset("ASSERT-KTH/DISL",'decomposed')  # Replace with your dataset name

# Ensure only the required columns are present and limit to 10,000 samples
train_dataset = dataset["train"].shuffle(seed=42).select(range(min(len(dataset["train"]), 10000)))
filtered_dataset = train_dataset.remove_columns(
    [col for col in train_dataset.column_names if col not in ["contract_address", "source_code"]]
)

# Clean and validate the dataset

if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})  # Add a [PAD] token
    print("Added a [PAD] token to the tokenizer.")

# Preprocessing function

from datasets import DatasetDict

# Subset the dataset to 10,000 samples before preprocessing
subset_dataset = dataset["train"].shuffle(seed=42).select(range(10000))

# Preprocessing function
from datasets import DatasetDict

# Remove irrelevant columns before preprocessing
filtered_dataset = dataset["train"].shuffle(seed=42).select(range(10000))
filtered_dataset = filtered_dataset.remove_columns(
    [col for col in filtered_dataset.column_names if col not in ["contract_address", "source_code"]]
)

# Preprocessing function
def preprocess_function(examples):
    # Ensure inputs and outputs are strings
    inputs = [str(addr) for addr in examples["contract_address"]]
    outputs = [str(code) for code in examples["source_code"]]

    # Tokenize input and output columns
    tokenized_inputs = tokenizer(
        inputs,
        padding="max_length",
        truncation=True,
        max_length=128
    )
    tokenized_outputs = tokenizer(
        outputs,
        padding="max_length",
        truncation=True,
        max_length=512
    )

    # Replace pad tokens with -100 for labels
    labels = [
        [(token if token != tokenizer.pad_token_id else -100) for token in label]
        for label in tokenized_outputs["input_ids"]
    ]

    return {
        "input_ids": tokenized_inputs["input_ids"],
        "attention_mask": tokenized_inputs["attention_mask"],
        "labels": labels,
    }

# Apply preprocessing
tokenized_dataset = filtered_dataset.map(preprocess_function, batched=True)



# Split the dataset into train and test sets (90% train, 10% test)
dataset_split = tokenized_dataset.train_test_split(test_size=0.1, seed=42)
train_dataset = dataset_split["train"]
eval_dataset = dataset_split["test"]

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="steps",
    eval_steps=500,
    save_steps=500,
    logging_dir="./logs",
    per_device_train_batch_size=2,  # Adjust batch size for your system
    num_train_epochs=3,
    weight_decay=0.01,
    learning_rate=5e-5,
    remove_unused_columns=False,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
)

# Train the model
trainer.train()



Added a [PAD] token to the tokenizer.


Resolving data files:   0%|          | 0/18 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/18 [00:00<?, ?it/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-4-55a440f2ad39>:100: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length. Perhaps your features (`contract_address` in this case) have excessive nesting (inputs type `list` where type `int` is expected).

In [ ]:
# Save the model and tokenizer
model.save_pretrained("./llama-3.2-1b-finetuned")
tokenizer.save_pretrained("./llama-3.2-1b-finetuned")


In [ ]:
# Define the save directory for the fine-tuned model and tokenizer
save_directory = "./meta-llama-3.2-1B-finetuned"

# Save the fine-tuned model
model.save_pretrained(save_directory)

# Save the tokenizer
tokenizer.save_pretrained(save_directory)

print(f"Model and tokenizer saved to {save_directory}")
